In [122]:
from __future__ import division
from __future__ import print_function
from PIL import Image
from StringIO import StringIO
import urllib2
from urlparse import urlparse
import sys
import os
import numpy

In [123]:
def process_image(image, blocks=4):
    if not image.mode == 'RGB':
        return None
    feature = [0] * blocks * blocks * blocks
    pixel_count = 0
    for pixel in image.getdata():
        ridx = int(pixel[0]/(256/blocks))
        gidx = int(pixel[1]/(256/blocks))
        bidx = int(pixel[2]/(256/blocks))
        idx = ridx + gidx * blocks + bidx * blocks * blocks
        feature[idx] += 1
        pixel_count += 1
    return [x/pixel_count for x in feature]

In [124]:
def process_image_file(image_path):
    image_fp = StringIO(open(image_path, 'rb').read())
    try:
        image = Image.open(image_fp)
        return process_image(image)
    except IOError:
        return None

In [125]:
def process_directory(directory):
    training = []
    for root, _, files in os.walk(directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            img_feature = process_image_file(file_path)
            if img_feature:
                training.append(img_feature)
    return training

In [182]:
def show_plot(features):
    for img in features:
        plt.plot(img)
        plt.show()

In [126]:
day = process_image_file('day_night/day.jpg')
night = process_image_file('day_night/night.jpg')

In [127]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

clf.fit([day,night], ['day', 'night'])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [128]:
n_t1 = process_image_file('day_night/nigt-test.png')
n_t2 = process_image_file('day_night/nigt-test2.jpg')
d_t1 = process_image_file('day_night/day-test.jpg')
d_t2 = process_image_file('day_night/day-test2.jpg')

In [129]:
clf.predict(d_t1)

/home/andreicaprar/miniconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array(['day'], 
      dtype='|S5')